# 課題番号4

* Name: Kadoi Takemaru
* ID: 03-190413

次のようなバケツパズルを解いてみよう。 番号を振った$n$個のバケツが順に並べてある。$i$番のバケツには$n+1-i$という番号が振られたボールが$2$つ入っている。 隣り合ったバケツから$1$つずつボールを取り出して交換するという操作を考える。 すべてのボールが自分と同じ番号のバケツに入るようにするのがゴールである。

$n=4$と$n=5$については、 `A*`,`深さ優先探索`,`広さ優先探索`をそれぞれ実行して探索の効率（解に到達するまでのノー ドの展開数）を比較すること。

## 簡単なコード説明

以下はバケツパズルを解くのに使用した関数である。それぞれのセルごとに`補助関数`、`深さ優先探索`、`幅優先探索`、`A*`のコードが書かれている。以下ではそれぞれのアルゴリズムの説明をする。

各アルゴリズムに共通して配列の$i$番目の要素は$i$が奇数のとき$2i+1$番目、$i$が偶数のとき$2i$番目のバケツのボールの番号である。

### 実行環境

`python3.8`以上でJupyterを使用した。

### 深さ優先探索

FIFOを用いる。
FIFOは最初、初期のバケツが入っており、隣接したバケツを探索し探索したノード(バケツ)を格納していく。

### 幅優先探索

LIFOを用いる。
LIFOは最初、初期のバケツが入っており、隣接したバケツを探索し探索したノード(バケツ)を格納していく。

### A*

ヒューリスティックにはマンハッタン距離を求める関数を用いる。

なお、アルゴリズムは[Wikipedia](https://ja.wikipedia.org/wiki/A*)と[こちらのページ](http://www.nct9.ne.jp/m_hiroi/light/pyalgo28.html)また、[最短経路探索ゼミ](http://bin.t.u-tokyo.ac.jp/tansaku_18/)のコードを参考にした。

In [118]:
# ライブラリ
from collections import deque
from collections import defaultdict
import heapq

# 補助関数
# パズルの答えを作る
def create_answer(n):
    result = []
    for i in range(n):
        result.append(i+1)
        result.append(i+1)
    return result

# 与えられるバケツを作る
def create_bucket(n):
    return create_answer(n)[::-1]

# バケツの玉を交換する際の添字を作る
def create_swap_list(n):
    result = []
    for i in range(2*n - 2):
        if i % 2 == 0:
            result.append((i, i+2))
            result.append((i, i+3))
        else:
            result.append((i, i+1))
            result.append((i, i+2))
    return result

# バケツの配列がどのような推移をしたか配列で返す
def find_transition(l, dic):
    cur = l.copy()
    result = [cur]
    while True:
        hash_key = "".join(map(str, cur))
        if hash_key in dic:
            cur = dic[hash_key]
            result.append(cur)
        else:
            return result[::-1]

# 2つの配列のマンハッタン距離を求める
def calc_manhathan(l, answer):
    return sum([e1 != e2 for e1, e2 in zip(l, answer)])

In [ ]:
# 深さ優先探索
def dfs(n):
    bucket = create_bucket(n)
    answer = create_answer(n)
    swap_list = create_swap_list(n)
    total_step = 0
    step_of_bucket = {}
    step_of_bucket["".join(map(str, bucket))] = 0
    parent_of_bucket = {}
    stack = [bucket]
    while stack:
        cur = stack.pop()
        cur_step = step_of_bucket["".join(map(str, cur))]
        if cur == answer:
            print(f"DFS: {total_step=}, {cur_step=}")
            print(f"Trainsition sequence: {find_transition(cur, parent_of_bucket)}")
            return
        for (i, j) in swap_list:
            total_step += 1
            new_bucket = cur.copy()
            new_bucket[i], new_bucket[j] = new_bucket[j], new_bucket[i]
            if (hash_key := "".join(map(str, new_bucket))) not in step_of_bucket:
                parent_of_bucket[hash_key] = cur
                step_of_bucket[hash_key] = cur_step + 1
                stack.append(new_bucket)
    print("DFS: no answer")

In [ ]:
# 幅優先探索
def bfs(n):
    bucket = create_bucket(n)
    answer = create_answer(n)
    swap_list = create_swap_list(n)
    total_step = 0
    step_of_bucket = {}
    step_of_bucket["".join(map(str, bucket))] = 0
    parent_of_bucket = {}
    queue = deque([bucket])
    while queue:
        cur = queue.popleft()
        cur_step = step_of_bucket["".join(map(str, cur))]
        if cur == answer:
            print(f"BFS: {total_step=}, {cur_step=}")
            print(f"Trainsition sequence: {find_transition(cur, parent_of_bucket)}")
            return
        for (i, j) in swap_list:
            total_step += 1
            new_bucket = cur.copy()
            new_bucket[i], new_bucket[j] = new_bucket[j], new_bucket[i]
            if (hash_key := "".join(map(str, new_bucket))) not in step_of_bucket:
                parent_of_bucket[hash_key] = cur
                step_of_bucket[hash_key] = cur_step + 1
                queue.append(new_bucket)
    print("BFS: no answer")

In [ ]:
# A*
def astar(n):
    bucket = create_bucket(n)
    answer = create_answer(n)
    swap_list = create_swap_list(n)
    total_step = 0
    step_of_bucket = {}
    step_of_bucket[tuple(bucket)] = 0
    parent_of_bucket = {}
    close_set = [] # 計算済みのノード
    open_set = [bucket] # 計算中のノード
    g_score = defaultdict(lambda: float("inf")) # nodeまでのコストの推定値
    f_score = defaultdict(float) # nodeからゴールまでのコストの推定値
    while open_set:
        cur = min((f_score[tuple(node)], node) for node in open_set)[1]
        cur_step = step_of_bucket[tuple(cur)]
        if cur == answer:
            print(f"A*: {total_step=} {cur_step=}")
            print(f"Trainsition sequence: {find_transition(cur, parent_of_bucket)}")
            return
        open_set.remove(cur)
        close_set.append(cur)
        for (i, j) in swap_list:
            total_step += 1
            nei = cur.copy()
            nei[i], nei[j] = nei[j], nei[i]
            if nei in close_set:
                continue
            step_of_bucket[tuple(nei)] = cur_step + 1
            parent_of_bucket[tuple(nei)] = cur
            if nei not in open_set:
                step_of_bucket[tuple(nei)] = cur_step + 1
                open_set.append(nei)
            tentative_g_score = g_score[tuple(cur)] + 1
            if tentative_g_score >= g_score[tuple(nei)]:
                continue
            g_score[tuple(nei)] = tentative_g_score
            f_score[tuple(nei)] = g_score[tuple(nei)] + calc_manhathan(nei, answer)
    print("A*: no answer")

## n=4のときの交換操作列を求めよ。

解に到達するまでのノードの展開数は以下の通りとなった。

|DFS|BFS|A*|
|-|-|-|
|13440|30228|180|

一方、元のバケツから解のバケツまでに必要な遷移の仕方はそれぞれ下のような結果となる。

|DFS|BFS|A*|
|-|-|-|
|303|10|16|

下のセルを実行するとDFS、DFS、A*でのノードの展開数、解に到達するまでに必要な遷移数、その際の遷移列がわかる

In [ ]:
dfs(4)
bfs(4)
astar(4)

## n=5のときの交換操作列を求めよ。

解に到達するまでのノードの展開数は以下の通りとなった。

|DFS|BFS|A*|
|-|-|-|
|285952|1814384|416|

一方、元のバケツから解のバケツまでに必要な遷移の仕方はそれぞれ下のような結果となる。

|DFS|BFS|A*|
|-|-|-|
|17158|15|26|

下のセルを実行するとDFS、DFS、A*でのノードの展開数、解に到達するまでに必要な遷移数、その際の遷移列がわかる

In [ ]:
dfs(5)
bfs(5)
astar(5)

## n=6のときのできるだけ少ない回数の交換操作列を求めよ。

次のセルを実行することで交換操作列を求めることができる。

BFSを用いた結果、ノードの展開数は149687980、解までに必要な遷移数は23とわかった。

In [ ]:
bfs(6)

## 考察

上記の結果から今回の例では求める交換操作列を高速に求めたい場合は`A*`、少ない回数の交換操作列を求めたい場合は`BFS`を使うべきだということが推測できる。`DFS`は今回の例では`A*`の下位互換という結果になった。

また、速度と得られる交換操作列の回数の少なさはトレードオフの関係にあり、`BFS`では高速に探索することは困難なので共に高い値を得るためには`A*`のヒューリスティックを適切に選ぶ必要があると考えられる。